In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
from tqdm import tqdm
import numpy as np
from sklearn.decomposition import MiniBatchDictionaryLearning
from sklearn.preprocessing import LabelEncoder
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Funções

In [2]:
def metrica_avaliacao(df, real_columns = 'acidentes_graves', predict_column='risk_index', viaturas=2, number_random_executions=1):

    sum_true = 0
    sum_pred = 0
    
    for month in df.month_number.unique():
        df_m = df[df.month_number==month]
        for dia in df_m.dia.unique():
            df_d = df_m[df_m.dia==dia]
            for p in df_d.periodo.unique():
                df_p_real = df_d[df_d.periodo == p].copy()
                df_p_pred = df_p_real.copy()
                
                sum_true += df_p_real.sort_values(by='acidentes_graves', ascending=False)['acidentes_graves'][:viaturas].sum()
                sum_pred += df_p_pred.sort_values(by=predict_column, ascending=False)['acidentes_graves'][:viaturas].sum()
                
    return sum_true, sum_pred

# Read Data Orig

In [3]:
df = pd.read_csv('data/partial_pedataset.csv')

clima_replace = {
    'sol': 'tempo_claro',
    'céu_claro': 'tempo_claro',
    'ignorado': 'ignorado',
    'chuva': 'tempo_adverso',
    'nublado': 'tempo_adverso',
    'nevoeiro_neblina': 'tempo_adverso'
}

for d in clima_replace:
    df.condicao_meteorologica.replace(to_replace=d,
                                      value=clima_replace[d],
                                      inplace=True)

print(df.condicao_meteorologica.unique())

df.drop(['mes'], axis=1, inplace=True)
df['dia'] = [int(str(d)[8:10]) for d in pd.to_datetime(df['timestamp'])]
df['data'] = pd.to_datetime(df.timestamp, yearfirst=True, errors='ignore')
df.drop([
    'timestamp', 'municipio', 'tipo_acidente', 'classificacao_acidente', 'fase_dia',
    'sentido_via', 'pessoas', 'label','causa_acidente','uf'],axis=1, inplace=True)

df = df[((df['br']==104) | (df['br']==232) & (df['km']>=102)& (df['km']<=213)) & (df['ano']>2016)]
df['acidentes_graves'] = df['mortos']+df['feridos_graves']
df['acidentes_graves'].apply(lambda x: 1 if (x > 0) else 0)
df.head()

['tempo_claro' 'ignorada' 'tempo_adverso']


,dia_semana,br,km,condicao_meteorologica,tipo_pista,tracado_via,uso_solo,mortos,feridos_leves,feridos_graves,...,feridos,veiculos,feriado,km_bins,periodo,ano,month_number,dia,data,acidentes_graves
62572,domingo,104.0,48.4,tempo_claro,dupla,reta,não,0,1,0,...,1,1,not-feriado,"[45, 50)","[16, 20)",2017,1,1,2017-01-01 19:35:00,0
62575,domingo,232.0,142.0,tempo_claro,dupla,reta,não,0,0,0,...,0,2,not-feriado,"[140, 145)","[20, 24)",2017,1,1,2017-01-01 22:00:00,0
62583,segunda,232.0,201.0,tempo_claro,simples,reta,não,0,0,0,...,0,6,not-feriado,"[200, 205)","[16, 20)",2017,1,2,2017-01-02 17:00:00,0
62600,quarta,232.0,184.8,tempo_claro,simples,desvio temporário,não,0,0,0,...,0,2,not-feriado,"[180, 185)","[8, 12)",2017,1,4,2017-01-04 10:00:00,0
62621,sexta,104.0,71.0,tempo_claro,dupla,curva,não,0,0,0,...,0,1,not-feriado,"[70, 75)","[20, 24)",2017,1,6,2017-01-06 20:55:00,0


# Read Padded

In [4]:
df_tudo_pad = pd.read_csv('df_19_18_17_16_pad.csv')

In [5]:
df_tudo_pad.columns

Index(['index', 'dia_semana', 'br', 'km', 'condicao_meteorologica',
       'tipo_pista', 'tracado_via', 'uso_solo', 'mortos', 'feridos_leves',
       'feridos_graves', 'ilesos', 'ignorados', 'feridos', 'veiculos',
       'feriado', 'km_bins', 'periodo', 'ano', 'month_number', 'dia', 'data'],
      dtype='object')

In [6]:
df_tudo_pad['acidentes_graves'] = df_tudo_pad['mortos'
                                    ]+df_tudo_pad['feridos_graves'
                                    ]+df_tudo_pad['feridos'
                                    ]+df_tudo_pad['feridos_leves'
                                    ]+df_tudo_pad['ilesos']

df_tudo_pad['acidentes_graves'] = df_tudo_pad['acidentes_graves'].apply(lambda x: 1 if (x > 0) else 0)

In [7]:
df_tudo_pad['acidentes_graves'].unique()

array([0, 1], dtype=int64)

In [8]:
df_tudo_pad['acidentes_graves'].value_counts()

0    431471
1      2149
Name: acidentes_graves, dtype: int64

In [9]:
df_tudo_pad.drop(['index','km'], axis=1, inplace=True)
df_tudo_pad.head()

,dia_semana,br,condicao_meteorologica,tipo_pista,tracado_via,uso_solo,mortos,feridos_leves,feridos_graves,ilesos,...,feridos,veiculos,feriado,km_bins,periodo,ano,month_number,dia,data,acidentes_graves
0,terca,104,ignorada,dupla,reta,sim,0,0,0,0,...,0,0,not-feriado,"[60, 65)","[8, 12)",2019,1,1,2019-01-01 00:00:00,0
1,terca,104,ignorada,dupla,reta,sim,0,0,0,0,...,0,0,not-feriado,"[60, 65)","[16, 20)",2019,1,1,2019-01-01 00:00:00,0
2,terca,104,ignorada,dupla,reta,sim,0,0,0,0,...,0,0,not-feriado,"[60, 65)","[20, 24)",2019,1,1,2019-01-01 00:00:00,0
3,terca,104,ignorada,dupla,reta,sim,0,0,0,0,...,0,0,not-feriado,"[60, 65)","[12, 16)",2019,1,1,2019-01-01 00:00:00,0
4,terca,104,ignorada,dupla,reta,sim,0,0,0,0,...,0,0,not-feriado,"[60, 65)","[4, 8)",2019,1,1,2019-01-01 00:00:00,0


# Train

## Encoding

In [10]:
df_enc = df_tudo_pad.copy()

# Categorical boolean mask
categorical_feature_mask = df_enc.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = df_enc.columns[categorical_feature_mask].tolist()

les = []

# import labelencoder
# instantiate labelencoder object

for cat_col in categorical_cols:

    le = LabelEncoder()
    le.fit(df_enc[cat_col])

    # apply le on categorical feature columns
    df_enc[cat_col] = le.transform(df_enc[cat_col])
    
    les.append(le)

In [11]:
df_enc.head()

,dia_semana,br,condicao_meteorologica,tipo_pista,tracado_via,uso_solo,mortos,feridos_leves,feridos_graves,ilesos,...,feridos,veiculos,feriado,km_bins,periodo,ano,month_number,dia,data,acidentes_graves
0,7,104,0,0,6,2,0,0,0,0,...,0,0,1,35,5,2019,1,1,2404,0
1,7,104,0,0,6,2,0,0,0,0,...,0,0,1,35,2,2019,1,1,2404,0
2,7,104,0,0,6,2,0,0,0,0,...,0,0,1,35,3,2019,1,1,2404,0
3,7,104,0,0,6,2,0,0,0,0,...,0,0,1,35,1,2019,1,1,2404,0
4,7,104,0,0,6,2,0,0,0,0,...,0,0,1,35,4,2019,1,1,2404,0


### unenconding

In [12]:
# for le, cat_col in enumerate(categorical_cols):

#     # apply le on categorical feature columns
#     df_enc[cat_col] = les[le].inverse_transform(df_enc[cat_col])
    
# df_enc.head()

## balancing

In [13]:
X_columns = ['dia_semana', 'br', 'condicao_meteorologica', 'tipo_pista','tracado_via', 'uso_solo','km_bins',
             'periodo', 'month_number', 'dia']

df_train = df_enc[(df_tudo_pad.ano== 2018) | (df_tudo_pad.ano== 2017)].copy()

g = df_train.groupby('acidentes_graves')
g = pd.DataFrame(g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True)))
df_train_balanced = g.reset_index(drop=True)
        
X_train_unbalanced = df_train[X_columns]

X_train_balanced = df_train_balanced[X_columns]       
y_train_balanced = df_train_balanced['acidentes_graves']

In [14]:
y_train_balanced.value_counts()

1    1080
0    1080
Name: acidentes_graves, dtype: int64

In [15]:
df_test = df_enc[(df_tudo_pad.ano == 2019)].copy()

g = df_test.groupby('acidentes_graves')
g = pd.DataFrame(g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True)))
df_test_balanced = g.reset_index(drop=True)

X_test_balanced = df_test_balanced[X_columns]
y_test_balanced = df_test_balanced['acidentes_graves']

X_test_unbalanced = df_test[X_columns]
y_test_unbalanced = df_test['acidentes_graves']

In [16]:
y_test_balanced.value_counts()

1    353
0    353
Name: acidentes_graves, dtype: int64

In [17]:
y_test_unbalanced.value_counts()

0    102577
1       353
Name: acidentes_graves, dtype: int64

## Fitiing

In [30]:
# transformer = MiniBatchDictionaryLearning(n_components=10)
# transformer.fit(X_train_balanced)

# X_pca = transformer.transform(X_train_balanced)
# print(X_pca.shape)

(2160, 10)


In [40]:
clf = xgb.XGBClassifier(objective ='reg:squarederror', colsample_bytree = 1, n_estimators = 200)
clf.fit(X_train_balanced, y_train_balanced)
y_train_pred = clf.predict(X_train_balanced)

# Test

In [41]:
accuracy_score(y_train_pred, y_train_balanced)

0.8953703703703704

In [43]:
y_test_pred = clf.predict(X_test_balanced)
accuracy_score(y_test_pred, y_test_balanced)

0.8512747875354107

In [45]:
y_test_pred_proba = clf.predict_proba(X_test_balanced)

In [49]:
df_test['proba_class_acc'] = clf.predict_proba(X_test_unbalanced)[:,1]

In [50]:
metrica_avaliacao(df_test, predict_column='proba_class_acc', viaturas = 3)

(353, 145)

In [28]:
for le, cat_col in enumerate(categorical_cols):

    # apply le on categorical feature columns
    df_test[cat_col] = les[le].inverse_transform(df_test[cat_col])
    
df_test.head()

,dia_semana,br,condicao_meteorologica,tipo_pista,tracado_via,uso_solo,mortos,feridos_leves,feridos_graves,ilesos,...,veiculos,feriado,km_bins,periodo,ano,month_number,dia,data,acidentes_graves,proba_class_acc
0,terca,104,ignorada,dupla,reta,sim,0,0,0,0,...,0,not-feriado,"[60, 65)","[8, 12)",2019,1,1,2019-01-01 00:00:00,0,0.66497
1,terca,104,ignorada,dupla,reta,sim,0,0,0,0,...,0,not-feriado,"[60, 65)","[16, 20)",2019,1,1,2019-01-01 00:00:00,0,0.66497
2,terca,104,ignorada,dupla,reta,sim,0,0,0,0,...,0,not-feriado,"[60, 65)","[20, 24)",2019,1,1,2019-01-01 00:00:00,0,0.66497
3,terca,104,ignorada,dupla,reta,sim,0,0,0,0,...,0,not-feriado,"[60, 65)","[12, 16)",2019,1,1,2019-01-01 00:00:00,0,0.66497
4,terca,104,ignorada,dupla,reta,sim,0,0,0,0,...,0,not-feriado,"[60, 65)","[4, 8)",2019,1,1,2019-01-01 00:00:00,0,0.66497


In [29]:
df_test.to_excel('epa.xlsx')